# Notas sobre punto flotante

Con el propósito de reforzar el tema sobre conversión de números a punto flotante y viceversa utilizando el esquema IEEE-754 les dejo estas notas para que no queden lagunas al respecto.

## Conversión de decimal a flotante

Para ello debemos saber convertir enteros con punto decimal en binario, para ello se utilizan divisiones y multiplicaciones en base 2. Habrán dos formas de conversión una para 32 bits y otra para 64 bits, es preciso recordar que la capacidad de memoria en variables de 64 bits es mucho mayor a la de 32 bits, por lo que se verá reflejado en la precisión. La conversión se compone de 3 partes:

* El signo (1bit para ambos casos)
* El exponente (8 bits para float32 y 11 bits para float64)
* La mantisa (23 bits restantes para float32 y 52 bits restantes para float64)

En este caso los bits corresponden a digitos de $1$s y $0$s. Para el caso del signo es muy simple si el número decimal es mayor a cero entonces se fija el bit de signo en `0` mientras que en el caso contrario el bit de signo es `1`.

### Primer paso: Normalizar el número decimal en base 2

Normalmente la conversión a punto flotante comienza con la conversión a binario de la parte entera y la parte fraccionaria, pero habrán casos en que la parte entera sea de un orden de magnitud grande (o muy grande) o que la parte fraccionaria tenga un orden de maginitud pequeño (o muy pequeño). En estos casos es engorrosa y talachuda la conversión de ambas partes; esto se puede solucionar *normalizando* al número decimal, es decir, convirtiéndolo en su parte más fundamental equivalente a binario pero aún siendo decimal. Ustedes recordarán que al normalizar cierta cantidad ya sea que hablemos de un vector, una distribución, un conjunto, etc, se debe de considerar la cantidad a normalizar y dividirla entre su magnitud; la *norma* en vectores, la *desviación estándar* en una distribución, la *suma* de todos los elementos en un conjunto por ejemplo. En este caso nos enfocamos en el exponente de base $2$ que se traduce en el orden de magnitud en el que se encuentra el número decimal al que vamos a normalizar
$$
\begin{split}
n=\log_2(\text{num}),\qquad\text{de tal forma que}\qquad \text{num}_{normalizado} &= \frac{\text{num}}{2^n}
\end{split}
$$
Este paso es sumamente importante ya que de esta forma te ahorras la conversión de la parte entera y te concentras en la conversión de la parte fraccionaria. Ahora haciendo esto se mata dos pájaros de un tiro porque así mismo $n$ es el exponente que se necesita para determinar el exponente en binario.

### Segundo paso: Conversión del exponente

Como se dijo anteriormente, se ocupa $n$ para determinar el exponente en binario. Para determinar el exponente binario se considera lo que se conoce como *sesgo* que es un valor fijo de $127$ y que corresponde con el exponente $0$ en decimal. Para el exponente binario se realiza la siguiente suma
$$
\text{exp}_{\text{bin}}=127+n
$$
Nota que $n$ en realidad puede ser positivo o negativo dependiendo del orden de magnitud que se quiera convertir. El resultado de $\text{exp}_{\text{bin}}$ sigue siendo un número decimal que se debe de convertir en base 2 utilizando el algoritmo de las divisiones entre dos; además el exponente debe ser capaz de poderse representar en 8 bits u 8 digitos que corresponde a la sección del exponente en binario.

### Tercer paso: Determinar la mantisa

Suponiendo que ya tengas tu conversión del exponente en binario ($\text{exp}_2$) falta determinar la mantisa como último elemento de la lista. Para ello vamos a tomar la parte fraccionaria del número normalizado (es decir de $\text{num}_{normalizado}-1$) y vamos a convertir ese número decimal en base 2. Para ello se deben de realizar continuas multiplicaciones por $2$, de tal forma que si se tiene un 1 a la izquierda del punto, se agrega un 1 a tu secuencia de binarios y en caso contrario se agrega un 0.

### Veamos un ejemplo: $k_b=1.380649^{-23}$

De entrada sabemos que el bit de signo es $0$ puesto que el número es mayor que cero, por lo que procedemos con la normalización del número:
$$
n=\log_2(kb)\approx -76
$$
es convieniente redondear y en su algoritmo incluir el método `np.round()` para que tengan un exponente entero al que convertirán en binario. Por lo tanto el número normalizado es $k_{bn} = \frac{k_b}{2^{(-76)}}\approx 1.043188889953199$; este número normalizado es al que le quitaremos su parte entera y nos enfocaremos en su parte fraccionaria para obtener la mantisa. Antes de pasar a la mantisa obtenemos la representación del exponente en decimal: $127+(-76)=51$, este número lo convertimos en binario y nos da como resultado: $00110011_2$. Ahora para obtener la mantisa consideramos $k_{bn}-1$ (es decir la parte fraccionaria del número decimal normalizado) y realizamos la conversión a binario que es $00001011000011100110110_2$. Y con esto tenemos finalmente el resultado esperado:

```0 00110011 00001011000011100110110```

Te encomiendo cada una de las cuentas realizadas para este ejercicio 😉 

## Conversión de flotante a decimal

Este caso es mucho más sencillo, en esencia se debe de aplicar la siguiente fórmula:
$$
N = (-1)^s\times 2^{\text{exp}_{10}}\times (1+M_{10})
$$
donde $s$ es el signo (el primer bit), $\text{exp}_{10}$ corresponde al exponente convertido en base 10 y la $M_{10}$ es la mantisa convertida en base 10. Para poder determinarlos hay que realizar las correspondientes multiplicaciones $2^j$ donde $j$ es la posición en donde te encuentras (y en donde existe un 1 por supuesto); ya sea a la izquierda del punto decimal que va de $0$ a $7$ (para este caso particular), o a la derecha del punto decimal que se consdiera de $-1$ a $-23$.

Por último les comparto mi forma de generar el algoritmo de conversión para el caso particular de 32 bits, les encomiendo mucho extender el algoritmo para el caso de 64 bits (únicamente deben de generalizar algunos puntos enfocados en el número de bits en el exponente y el número de bits en la mantisa).

In [1]:
import numpy as np 

def convert_toFloat(num):
    """
    num := número decimal en cualquier formato: real o notación científica
    """
    #Se determina el signo, con operador ternario.
    signo = "0" if num > 0 else '1'

    #Se normaliza el número ingresado para determinar el exponente y la mantisa
    normalize = np.round(np.log2(num))
    num = num/2**(normalize)

    #Se determina el exponente
    exp = 127 + normalize
    exponente = " "*8
    list_exponente = list(exponente)
    #Se genera una lista de 8 espacios para guardar en sentido inverso cada 1 o 0
    for i in range(7,-1,-1):
        exp /= 2
        if exp.is_integer():
            list_exponente[i] = '0'
        else:
            list_exponente[i] = '1'
            exp = int(exp) #importante convertir a entero para la siguiente iteración
    #Método join() para unir la lista de strings en uno solo
    exponente = "".join(list_exponente)       

    #Se determina la mantisa. Nos quedamos con la parte fraccionaria del número ya normalizado.
    c = num-1
    mantisa = " "*23
    list_mantisa = list(mantisa)
    #Se genera una lista de 23 espacios para guardar en cada uno un valor de la mantisa
    for i in range(23):
        c *= 2
        if c > 1:
            list_mantisa[i] = '1'
            c -= 1  #importante quitar la parte entera en este caso.
        else:
            list_mantisa[i] = '0'    
    #Método join() para unir las strings de la lista en una sola.
    mantisa = "".join(list_mantisa)
    
    return signo+exponente+mantisa

In [2]:
def convert_toDecimal(string):
    """
    string := es una cadena de números de longitud 32
    """
    #Si no se ingresa una cadena con dichas características no se ejecuta el algoritmo.
    if len(string) != 32:
        raise ValueError("El número debe de ser de 32 bits")

    #Se descompone la string en partes
    signo = int(string[0])
    exponente = string[1:9]
    mantisa = string[9:32]

    #Al segmento del exponente lo volvemos una lista para poder operar
    exponente_list = list(exponente)
    exp = 0
    for i in range(8):
        if exponente_list[7-i] == '1': #Vamos leyendo cada entrada de la lista en sentido inverso
            exp += 2**i
        else:
            continue
    #Se determina el exponente decimal
    exp = exp - 127    

    #Al segmento de la mantisa la volvemos una lista para poder operar
    mantisa_list = list(mantisa)
    man = 0
    for i in range(23):
        if mantisa_list[i] == '1':
            man += 1*2**(-i-1) #le restamos -1 al iterador para que quede acoplado
        else:
            continue
    #Se obtiene el número decimal con los resultados de cada ciclo for.
    return (-1)**signo*(2**exp)*(1+man)

In [3]:
kb = 1.380649e-23
e_float = "01000000001011011111100001010100"
kb_float = convert_toFloat(kb)
e = convert_toDecimal(e_float)
print("kb en punto flotante es: ",convert_toFloat(kb))
print("kb_float en decimal es: ",convert_toDecimal(kb_float))
print("")
print("e_float en decimal es: ",convert_toDecimal(e_float))
print("e en punto flotante es: ",convert_toFloat(e))

kb en punto flotante es:  00011001100001011000011100110110
kb_float en decimal es:  1.3806488946440725e-23

e_float en decimal es:  2.7182817459106445
e en punto flotante es:  01000000001011011111100001010011


Se puede notar una peuqeña variación en las últimas posiciones de cada conversión y ello puede deberse a un tema de redondeo (por ahora no estoy 100% seguro).